Perfect scores were seen on Kaggle's public leaderboards [before][1], but with a different evaluation function, and the authors never published their approach.

Someone else commented that a perfect score was obtainable
 in this competition after 198 submissions, or, since 3 submissions are allowed per day, 66 days,
 using a "brute force" method:


> *"The whole process takes about 198/3 = 66 days, which is shorter than the competition length."*



I wanted to see how quickly this could really be done, within the rules,
 and to try to win the race to the top of the public leaderboard.

I ended up using 14 submissions, or 5 days 
(Additionally,  I needlessly wasted 1 submission on the "All 0.5 Benchmark", 
 and spent another to actually claim the 0.00000 score, both being technically uninformative)

![leaderboard screenshot][2]

My approach is informed by information theory. You see, when the Kaggle server gives your submission a 
score, it emits up to 21.7  bits of information
 about the test labels, but there are only 198 labels with even fewer bits of information in them,
 so one could learn all there is to know about the labels in 8 submissions or so. Well, that's a
 theoretical limit, and might not be achievable in practice.

If you train any two models and choose the one that has a better leaderboard score, you are already using 1 bit of information from your public scores. A generalization of this to any number of models is [the boosting attack][3]. However, it would require 4 years to get to the perfect score here. My approach is fundamentally similar, but is much more effective, as it learns more bits from each score.

The core algebraic insight needed here is that if we choose 15 probabilities to be

`sigmoid(- n * epsilon * 2 ** i)` 

where n=198, 0 <= i < 15, and epsilon = 1.05e-5 for example, and choose the rest of the probabilities to be 0.5, then the 15 labels corresponding to those 15
probabilities are easily discoverable from the score we get, because all 
32768 possible label combinations lead to different scores.

Note that the final rankings are based on the **private** labels of the second stage. Discovering all **public** labels helps with those only indirectly, by effectively increasing your training set size by 14%.
(I believe the extra 14% are likely critical, given how close Kaggle competitions tend to be)

**USAGE**

To use the script, create an empty file called "scores.txt", copy "stage1_sample_submission.csv" (used to read patient IDs) into the same directory, and create a subdirectory called "submissions".

![tree_structure][4]

The former should contain the scores the Kaggle server gives you, one per line. It should be empty in the beginning. For example, the first line should be the score corresponding to "submission_00.csv". Keep any trailing 0s. There should be 5 digits after the decimal point.

You can rerun the script whenever you update "scores.txt", but it's not necessary. This will do some partial label inference. When that file contains 14 lines, rerunning the script should also generate "submission_fin.csv", which will have all the correct labels. ([Don't submit it though.][5] If you wish to verify the labels, you may want to submit *1-labels* instead and get the worst score possible: 34.54)



  [1]: https://www.kaggle.com/c/restaurant-revenue-prediction/forums/t/13950/our-perfect-submission
  [2]: http://i.imgur.com/na6J44f.png
  [3]: http://machinelearning.wustl.edu/mlpapers/papers/icml2015_blum15
  [4]: http://i.imgur.com/xufW4wV.png
  [5]: https://www.kaggle.com/c/data-science-bowl-2017/discussion/28597 

In [ ]:
from __future__ import print_function # for Python3.x compatibility
import numpy as np
from math import log


# INPUT / OUTPUT

def read_patient_ids():
    with open('stage1_sample_submission.csv') as f:
        lines = f.readlines()[1:]
        return [line.split(',')[0] for line in lines]

def prob_format(p):
    return '%e' % p

def truncate(p):
    return float(prob_format(p))

def write_submit(patient_ids, probs, file_name):
    assert len(patient_ids) == len(probs)
    with open(file_name, 'w') as f:
        f.write('id,cancer\n')
        for i, p in zip(patient_ids, probs):
            f.write('%s,%s\n' % (i, prob_format(p)))
    print('wrote %s' % file_name)

def read_scores():
    lines = open('scores.txt').readlines()
    return [s.strip() for s in lines]


# PROBABILITIES

def build_template(n, chunk_size):
    epsilon = 1.05e-5
    return 1 / (1 + np.exp(n * epsilon * 2 ** np.arange(chunk_size)))

def build_probs(n, chunk, template):
    assert template.shape == chunk.shape
    probs = np.zeros((n,))
    probs[:] = 0.5
    probs[chunk] = template
    return probs


# LABEL INFERENCE

def int_to_bin(x, size):
    s = bin(x)[2:][::-1].ljust(size, '0')
    return np.array([int(c) for c in s])

def update_labels(labels, chunk, template, score):
    assert template.shape == chunk.shape
    chunk_size = len(chunk)
    n = len(labels)
    match_count = 0
    for i in range(2**chunk_size):
        b = int_to_bin(i, chunk_size)
        score_i = ((-np.log(template) * b - np.log(1-template) * (1-b)).sum() \
                   - log(0.5) * (n-chunk_size))/n
        if score == ('%.5f' % score_i):
            match_count += 1
            new_labels = b
    assert match_count == 1 # no collisions
    print('new labels: %s' % new_labels)
    labels[chunk] = new_labels


# MAIN

def write_submit_files():
    n = 198
    np.random.seed(2017)
    idx = np.arange(n)
    np.random.shuffle(idx) # optional
    chunk_size = 15
    template = build_template(n, chunk_size)
    template = np.array([truncate(x) for x in template])

    scores = read_scores()
    labels = np.zeros((n,), dtype=np.int)
    labels[:] = -1

    patient_ids = read_patient_ids()
    chunks = [idx[i : i + chunk_size] for i in range(0, len(idx), chunk_size)]
    for i, chunk in enumerate(chunks):
        t = template[:len(chunk)]
        probs = build_probs(n, chunk, t)
        write_submit(patient_ids, probs, 'submissions/submission_%02d.csv' % i)
        if i < len(scores):
            update_labels(labels, chunk, t, scores[i])
            if i+1 == len(chunks):
                write_submit(patient_ids, labels, 'submissions/submission_fin.csv')

if __name__ == '__main__':
    write_submit_files()